In [ ]:
import pandas as pd
import plotly.express as px
import re

In [16]:
reviews = pd.read_json('..\\data\\Electronics_5.json', lines = True, nrows = 50000) # We read the json file croped in order to not surcharge the code
# Now let's save the extraced dataframe for future use
reviews.to_csv('..\\data\\electronics_reviews_50k.csv', index = False)
reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [17]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      50000 non-null  object
 1   asin            50000 non-null  object
 2   reviewerName    49706 non-null  object
 3   helpful         50000 non-null  object
 4   reviewText      50000 non-null  object
 5   overall         50000 non-null  int64 
 6   summary         50000 non-null  object
 7   unixReviewTime  50000 non-null  int64 
 8   reviewTime      50000 non-null  object
dtypes: int64(2), object(7)
memory usage: 3.4+ MB


In [18]:
# Let's first check the repartition in % of the overall grades 
counts = reviews['overall'].value_counts().sort_index()

fig = px.pie(
    names=counts.index,
    values=counts.values,
    title="Repartition of Overall Grades in %"
)

fig.show()

We can see that the database is unbalance with a majority of 5 grades and a minority of 1 - 2 stars
Now, to continue, let's just keep 2 columns : the reviews and the grade. The main objectif of this notebook will be to predict the grade
by the review.

In [19]:
# Just before going further, let's clean the dataset a bit by deleting the duplicated values.
# First let's see of we have such values (except in helpful )
duplicates = reviews.drop(columns=["helpful"]).duplicated()
print(f"There are {duplicates.sum()} duplicated rows in the dataset.")
#It seems that there are no duplicated rows in the dataset

There are 0 duplicated rows in the dataset.


In [20]:
reviews = reviews.loc[:, ["reviewText", "overall"]] # Select only the required columns for the analysis
reviews

,reviewText,overall
0,We got this GPS for my husband who is an (OTR)...,5
1,"I'm a professional OTR truck driver, and I bou...",1
2,"Well, what can I say. I've had this unit in m...",3
3,"Not going to write a long review, even thought...",2
4,I've had mine for a year and here's what we go...,1
...,...,...
49995,"I want to buy four more, now. This thing is in...",5
49996,price-quality ratio is very good. Colour and s...,3
49997,Really like this case.Before purchasing this o...,5
49998,What you don't find out until you get the pack...,2


In [21]:
reviews.isna().sum() # On voit qu'il n'y a aucune valeur NaN dans le dataset

reviewText    0
overall       0
dtype: int64

In [22]:
empty_text = reviews["reviewText"].str.strip().eq("").sum() # Check for empty strings
empty_grade = (reviews["overall"] == "").sum() # Check for empty grades
print(f"Not considering NaN values, there are still {empty_text} empty comments and {empty_grade} article without grade")
print(reviews[reviews["reviewText"].str.strip() == ""].head()) # We can display comments with empty strings 

Not considering NaN values, there are still 19 empty comments and 0 article without grade
     reviewText  overall
149                    5
293                    5
1212                   5
4633                   4
8246                   5


In [23]:
# Now let's delete the rows with empty strings in the reviewText column
reviews = reviews[~reviews["reviewText"].str.strip().eq("")]

In [ ]:
def clean_text(text):
    text = str(text).lower() # Convert to lowercase
    text = re.sub(r"[^a-zA-Z']", ' ', text) # Remove special characters and replace them by a space
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    return text

reviews['reviewText'] = reviews['reviewText'].apply(clean_text)
reviews.to_csv('..\\data\\electronics_reviews_cleaned_50k.csv', index = False)
reviews


C:\Users\alexi\AppData\Local\Temp\ipykernel_4608\3394670753.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,reviewText,overall
0,we got this gps for my husband who is an otr o...,5
1,i'm a professional otr truck driver and i boug...,1
2,well what can i say i've had this unit in my t...,3
3,not going to write a long review even thought ...,2
4,i've had mine for a year and here's what we go...,1
...,...,...
49995,i want to buy four more now this thing is incr...,5
49996,price quality ratio is very good colour and si...,3
49997,really like this case before purchasing this o...,5
49998,what you don't find out until you get the pack...,2
